In [1]:
import os
path = 'C:/pytest/'
os.chdir(path)

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [79]:
tf.config.run_functions_eagerly(True)

Instructions for updating:
Use `tf.config.run_functions_eagerly` instead of the experimental version.


In [3]:
import rhinoMorph
rn = rhinoMorph.startRhino()

filepath:  C:\Anaconda3\lib\site-packages
classpath:  C:\Anaconda3\lib\site-packages\rhinoMorph/lib/rhino.jar
RHINO started!


In [4]:
data = pd.read_excel('감성대화말뭉치(최종데이터)_Training.xlsx')

In [5]:
data_sent1 = data.loc[:,['연령','감정_대분류','사람문장1']]
data_sent1.columns = ['연령','감정_대분류','사람문장']
data_sent2 = data.loc[:,['연령','감정_대분류','사람문장2']]
data_sent2.columns = ['연령','감정_대분류','사람문장']
data_sent3 = data.loc[:,['연령','감정_대분류','사람문장3']]
data_sent3.columns = ['연령','감정_대분류','사람문장']
data_sent4 = data.loc[:,['연령','감정_대분류','사람문장4']]
data_sent4.columns = ['연령','감정_대분류','사람문장']

In [6]:
data_tot = pd.concat([data_sent1,data_sent2,data_sent3,data_sent4])

In [7]:
data_tot.dropna(inplace=True)

In [8]:
age_y = data_tot['연령']
emotion_y = data_tot['감정_대분류']

In [27]:
X = pd.read_excel('morphed_X.xlsx')['사람문장']

pandas.core.series.Series

In [42]:
# X = data_tot['사람문장']
age_y = data_tot['연령'].reset_index(drop = True)
emotion_y = data_tot['감정_대분류'].str.strip().reset_index(drop = True)

In [25]:
emotion_y.unique()

array(['기쁨', '불안', '당황', '슬픔', '분노', '상처'], dtype=object)

In [44]:
emotion_tot = pd.concat([X, emotion_y], axis = 1)
emotion_tot.head()

,사람문장,감정_대분류
0,아내 드디어 출산 되 정말 신 나,기쁨
1,당뇨 합병증 약이 가지 넘어가 스트레스,불안
2,고등학교 올라오 중학교 때 수업 갑자기 어렵 당황,당황
3,재취업 되 받 되 첫 월급 오 가족 외식 하 예정 너무 행복,기쁨
4,빚 드디어 다 갚 되 이제야 안도감 들,기쁨


In [45]:
emotion_excluded = emotion_tot[(emotion_tot.감정_대분류 !='당황') & (emotion_tot.감정_대분류 !='불안') & (emotion_tot.감정_대분류 !='상처')]
emotion_excluded['감정_대분류'].unique()

array(['기쁨', '슬픔', '분노'], dtype=object)

In [46]:
emotion_X  = emotion_excluded['사람문장']
emotion_y = emotion_excluded['감정_대분류']

In [10]:
# morphed_data = []
# for morphed_data_each in X:
#     morphed_text = rhinoMorph.onlyMorph_list(rn,morphed_data_each, pos = ['NNG','NNP','VV','VA','XR','IC','MM','MAG','MAJ'])
#     joined_text = ' '.join(morphed_text)
#     morphed_data.append(joined_text)

In [11]:
# morphed_X = pd.DataFrame(morphed_data, columns = ['사람문장'])
# morphed_X.to_excel('morphed_X.xlsx',index=False)

In [48]:
from sklearn.model_selection import train_test_split
# age
train_age_X, test_age_X, train_age_y, test_age_y = train_test_split(X, age_y,train_size=0.7, random_state=1111)
print(len(train_age_X), len(test_age_X))
# emotion
train_emo_X, test_emo_X, train_emo_y, test_emo_y = train_test_split(emotion_X, emotion_y,train_size=0.7, random_state=1111)
print(len(train_emo_X), len(test_emo_X))


80002 34287
40246 17249


In [50]:
text_len = [len(line.split(' ')) for line in train_age_X]
print('age\n최소길이 : {}\n최대 길이 : {}\n평균 길이 : {}\n중위수 길이 : {}\n구간별 최대 길이 : {}'.format(np.min(text_len), np.max(text_len), np.round(np.mean(text_len), 1), np.median(text_len), np.percentile(text_len, [0,25,50,75,99,100])))
text_len = [len(line.split(' ')) for line in train_emo_X]
print('emotion\n최소길이 : {}\n최대 길이 : {}\n평균 길이 : {}\n중위수 길이 : {}\n구간별 최대 길이 : {}'.format(np.min(text_len), np.max(text_len), np.round(np.mean(text_len), 1), np.median(text_len), np.percentile(text_len, [0,25,50,75,99,100])))

age
최소길이 : 1
최대 길이 : 30
평균 길이 : 7.9
중위수 길이 : 8.0
구간별 최대 길이 : [ 1.  6.  8. 10. 16. 30.]
emotion
최소길이 : 1
최대 길이 : 28
평균 길이 : 7.9
중위수 길이 : 8.0
구간별 최대 길이 : [ 1.  6.  8. 10. 16. 28.]


In [51]:
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
max_words = 10000
maxlen = 16

In [52]:
# tokenizing
# age
tokenizer_age = Tokenizer(num_words= max_words)
tokenizer_age.fit_on_texts(train_age_X)
word_index_age = tokenizer_age.word_index
# emo
tokenizer_emo = Tokenizer(num_words= max_words)
tokenizer_emo.fit_on_texts(train_emo_X)
word_index_emo = tokenizer_emo.word_index

In [54]:
# sequencing
# age
train_age_X_sequencing = tokenizer_age.texts_to_sequences(train_age_X)
train_emo_X_sequencing = tokenizer_emo.texts_to_sequences(train_emo_X)

In [55]:
# padding
train_age_X_padding = pad_sequences(train_age_X_sequencing, maxlen=maxlen)

train_emo_X_padding = pad_sequences(train_emo_X_sequencing, maxlen=maxlen)

In [56]:
# OHE
def to_one_hot(sequences, dimension):
    results = np.zeros((len(sequences),dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1
    return results

In [20]:
# train_emo_X_1hot = to_one_hot(train_emo_X_padding, dimension= max_words)
# train_emo_X_1hot

array([[1., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 1., ..., 0., 0., 0.],
       [1., 1., 0., ..., 0., 0., 0.]])

In [21]:
# train_emo_y = train_emo_y.str.strip()

In [22]:
# train_emo_y[train_emo_y == '불안 ']

Series([], Name: 감정_대분류, dtype: object)

In [60]:
from sklearn.preprocessing import LabelEncoder
e_age = LabelEncoder()
e_age.fit(train_age_y)
train_age_y_encoding = e_age.transform(train_age_y)
e_emo = LabelEncoder()
e_emo.fit(train_emo_y)
train_emo_y_encoding = e_emo.transform(train_emo_y)

In [61]:
train_age_y_encoding

array([2, 2, 2, ..., 0, 2, 2])

In [62]:
train_age_y_1hot = to_one_hot(train_age_y_encoding, dimension = train_age_y_encoding.max()+1)
train_emo_y_1hot = to_one_hot(train_emo_y_encoding, dimension = train_emo_y_encoding.max()+1)

In [29]:
len(train_emo_X_1hot)

80002

In [30]:
len(train_emo_y_1hot)

80002

In [64]:
train_train_age_X, train_valid_age_X, train_train_age_y, train_valid_age_y = train_test_split(train_age_X_padding, train_age_y_1hot, train_size = 0.7)
train_train_emo_X, train_valid_emo_X, train_train_emo_y, train_valid_emo_y = train_test_split(train_emo_X_padding, train_emo_y_1hot, train_size = 0.7)

In [90]:
embedding_dim = 200
epochs = 20
class_age_number = 4
class_emo_number = 3
batch_size = 100

In [91]:
from keras.models import Sequential
from keras.layers import Dense, Embedding,Flatten
model_age = Sequential()
model_age.add(Embedding(input_dim= max_words, output_dim= embedding_dim, input_length = maxlen))
model_age.add(Flatten())
model_age.add(Dense(64, activation = 'relu'))
model_age.add(Dense(32, activation = 'relu'))
model_age.add(Dense(class_age_number, activation = 'softmax'))
model_age.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 16, 200)           2000000   
                                                                 
 flatten_2 (Flatten)         (None, 3200)              0         
                                                                 
 dense_6 (Dense)             (None, 64)                204864    
                                                                 
 dense_7 (Dense)             (None, 32)                2080      
                                                                 
 dense_8 (Dense)             (None, 4)                 132       
                                                                 
Total params: 2,207,076
Trainable params: 2,207,076
Non-trainable params: 0
_________________________________________________________________


In [92]:
model_emo = Sequential()
model_emo.add(Embedding(input_dim= max_words, output_dim= embedding_dim, input_length = maxlen))
model_emo.add(Flatten())
model_emo.add(Dense(64, activation = 'relu'))
model_emo.add(Dense(32, activation = 'relu'))
model_emo.add(Dense(class_emo_number, activation = 'softmax'))
model_emo.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 16, 200)           2000000   
                                                                 
 flatten_3 (Flatten)         (None, 3200)              0         
                                                                 
 dense_9 (Dense)             (None, 64)                204864    
                                                                 
 dense_10 (Dense)            (None, 32)                2080      
                                                                 
 dense_11 (Dense)            (None, 3)                 99        
                                                                 
Total params: 2,207,043
Trainable params: 2,207,043
Non-trainable params: 0
_________________________________________________________________


In [93]:
from keras.callbacks import EarlyStopping, ModelCheckpoint
model_age.compile(loss= 'categorical_crossentropy', optimizer = 'rmsprop', metrics = ['acc'])
model_emo.compile(loss= 'categorical_crossentropy', optimizer = 'rmsprop', metrics = ['acc'])


In [94]:
es = EarlyStopping(monitor = 'val_acc', patience = 2)
cp_age = ModelCheckpoint(filepath = 'C:/pytest/models/age_model03.h5', monitor = 'val_acc', verbose = 1, save_best_only= True)

cp_emo = ModelCheckpoint(filepath = 'C:/pytest/models/emotion_model03.h5', monitor = 'val_acc', verbose = 1, save_best_only= True)
# history = model.fit(train_train_X[:3000], train_train_y[:3000], epochs = 50, batch_size = 32, verbose = 1, callbacks = [es, cp], validation_data = (train_valid_X[:3000], train_valid_y[:3000]))
# history_dict = history.history

In [95]:
# with tf.device('/CPU:0'):
#     model.fit(train_train_X, train_train_y, epochs = 50, batch_size = 32, verbose = 1, callbacks = [es, cp], validation_data = (train_valid_X, train_valid_y))
model_age.fit(train_train_age_X, train_train_age_y, epochs = epochs, batch_size = batch_size, verbose = 1, callbacks = [es, cp_age], validation_data = (train_valid_age_X, train_valid_age_y))

Epoch 1/20
561/561 [==============================] - ETA: 0s - loss: 0.9991 - acc: 0.5520
Epoch 1: val_acc improved from -inf to 0.55539, saving model to C:/pytest/models\age_model03.h5
561/561 [==============================] - 16s 29ms/step - loss: 0.9991 - acc: 0.5520 - val_loss: 1.0114 - val_acc: 0.5554
Epoch 2/20
560/561 [============================>.] - ETA: 0s - loss: 0.7849 - acc: 0.6699
Epoch 2: val_acc improved from 0.55539 to 0.60135, saving model to C:/pytest/models\age_model03.h5
561/561 [==============================] - 17s 29ms/step - loss: 0.7849 - acc: 0.6699 - val_loss: 0.9136 - val_acc: 0.6013
Epoch 3/20
560/561 [============================>.] - ETA: 0s - loss: 0.6726 - acc: 0.7270
Epoch 3: val_acc did not improve from 0.60135
561/561 [==============================] - 17s 30ms/step - loss: 0.6726 - acc: 0.7270 - val_loss: 0.9680 - val_acc: 0.5905
Epoch 4/20
560/561 [============================>.] - ETA: 0s - loss: 0.5661 - acc: 0.7772
Epoch 4: val_acc did not i

In [96]:
model_emo.fit(train_train_emo_X, train_train_emo_y, epochs = epochs, batch_size = batch_size, verbose = 1, callbacks = [es, cp_emo], validation_data = (train_valid_emo_X, train_valid_emo_y))

Epoch 1/20
280/282 [============================>.] - ETA: 0s - loss: 0.8116 - acc: 0.6130
Epoch 1: val_acc improved from -inf to 0.67103, saving model to C:/pytest/models\emotion_model03.h5
282/282 [==============================] - 9s 31ms/step - loss: 0.8108 - acc: 0.6136 - val_loss: 0.7267 - val_acc: 0.6710
Epoch 2/20
280/282 [============================>.] - ETA: 0s - loss: 0.5960 - acc: 0.7484
Epoch 2: val_acc did not improve from 0.67103
282/282 [==============================] - 9s 31ms/step - loss: 0.5958 - acc: 0.7485 - val_loss: 0.7323 - val_acc: 0.6674
Epoch 3/20
282/282 [==============================] - ETA: 0s - loss: 0.4573 - acc: 0.8189
Epoch 3: val_acc did not improve from 0.67103
282/282 [==============================] - 9s 30ms/step - loss: 0.4573 - acc: 0.8189 - val_loss: 0.8248 - val_acc: 0.6590


In [83]:
import pickle
with open('C:/pytest/models/age_tokenizer.pickle', 'wb') as f:
    pickle.dump(tokenizer_age, f, protocol= pickle.HIGHEST_PROTOCOL)
with open('C:/pytest/models/emotion_tokenizer.pickle', 'wb') as f:
    pickle.dump(tokenizer_emo, f, protocol= pickle.HIGHEST_PROTOCOL)

In [84]:
# tokenizing
test_age_X_sequnecing = tokenizer_age.texts_to_sequences(test_age_X)
# padding
test_age_X_padding = pad_sequences(test_age_X_sequnecing, maxlen= maxlen)
# test_y label
test_age_y_encoding = e_age.transform(test_age_y)
# OHE
test_age_y_1hot = to_one_hot(test_age_y_encoding, dimension= test_age_y_encoding.max()+1)

# tokenizing
test_emo_X_sequnecing = tokenizer_emo.texts_to_sequences(test_emo_X)
# padding
test_emo_X_padding = pad_sequences(test_emo_X_sequnecing, maxlen= maxlen)

# test_y label
test_emo_y_encoding = e_emo.transform(test_emo_y)
# OHE
test_emo_y_1hot = to_one_hot(test_emo_y_encoding, dimension= test_emo_y_encoding.max()+1)

In [97]:
age_eval = model_age.evaluate(test_age_X_padding, test_age_y_1hot)
print('연령 : {:.2f}%'.format(age_eval[1]*100))

1072/1072 [==============================] - 12s 11ms/step - loss: 1.0035 - acc: 0.6042
연령 : 60.42%


In [98]:
emo_eval = model_emo.evaluate(test_emo_X_padding, test_emo_y_1hot)
print('감정_대분류 : {:.2f}%'.format(emo_eval[1]*100))

540/540 [==============================] - 6s 11ms/step - loss: 0.8140 - acc: 0.6658
감정_대분류 : 66.58%
